In [1]:
import numpy as np
import os
import csv
from random import random, sample, seed
from datetime import datetime
import psutil
import resource
import gc

import tensorflow as tf

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import text_to_word_sequence, Tokenizer
from tensorflow.keras import Input, Model, Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, concatenate, Activation
from tensorflow.keras.layers import Masking, Dropout, Reshape
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import CSVLogger, TensorBoard, Callback

/home/david/anaconda3/envs/capstone/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/david/anaconda3/envs/capstone/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/david/anaconda3/envs/capstone/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/david/anaconda3/envs/capstone/lib/pyt

In [2]:
# Constants
data_path = 'data/submissions.csv'
embeddings_path = 'data/glove.6B.50d.txt'
max_features = 40000
maxlen = 20 # max input length
batch_size = 32
embedding_dims = 50 # word embedding dim
meta_embedding_dims = 64 # metadata embedding dim
epochs = 10

In [20]:
titles = []
hours = []
minutes = []
dayofweeks = []
dayofyears = []
is_top_submission = []

max_rows = 16000

with open(data_path, 'r', encoding="latin1") as f:
    reader = csv.DictReader(f)
    i = 0
    for submission in reader:
#         if i >= max_rows:
#             break
#         i += 1
        titles.append(submission['title'])
        hours.append(submission['hour'])
        minutes.append(submission['minute'])
        dayofweeks.append(submission['dayofweek'])
        dayofyears.append(submission['dayofyear'])
        is_top_submission.append(submission['is_top_submission'])
            
titles = np.array(titles)
hours = np.array(hours, dtype=int)
minutes = np.array(minutes, dtype=int)
dayofweeks = np.array(dayofweeks, dtype=int)
dayofyears = np.array(dayofyears, dtype=int)
is_top_submission = np.array(is_top_submission, dtype=int)

In [4]:
# for i in range(max_rows//2, max_rows):
#     is_top_submission[i] = 0

In [21]:
word_tokenizer = Tokenizer(max_features)
word_tokenizer.fit_on_texts(titles)

In [6]:
titles_tf = word_tokenizer.texts_to_sequences(titles)
titles_tf = tf.keras.preprocessing.sequence.pad_sequences(titles_tf, maxlen)

In [7]:
embedding_vectors = {}

with open(embeddings_path, 'r') as f:
    for line in f:
        line_split = line.strip().split(" ")
        vec = np.array(line_split[1:], dtype=float)
        word = line_split[0]
        embedding_vectors[word] = vec

In [8]:
weights_matrix = np.zeros((max_features + 1, 50))

for word, i in word_tokenizer.word_index.items():

    embedding_vector = embedding_vectors.get(word)
    if embedding_vector is not None and i <= max_features:
        weights_matrix[i] = embedding_vector

In [9]:
# zero based year
dayofyears_tf = dayofyears - 1

In [10]:
tf.reset_default_graph()
# sess.close()
tf.keras.backend.clear_session()
sess = tf.InteractiveSession()

In [17]:
def build_model():
    titles_input = Input(shape=(maxlen,), name='titles_input')
    titles_embedding = Embedding(max_features + 1, embedding_dims, weights=[weights_matrix])(titles_input)
    titles_pooling = GlobalAveragePooling1D()(titles_embedding)
    
    aux_output = Dense(1, activation='sigmoid', name='aux_out')(titles_input)
    
    hours_input = Input(shape=(1,), name='hours_input')
    hours_embedding = Embedding(24, meta_embedding_dims)(hours_input)
    hours_reshape = Reshape((meta_embedding_dims,))(hours_embedding)

    dayofweeks_input = Input(shape=(1,), name='dayofweeks_input')
    dayofweeks_embedding = Embedding(7, meta_embedding_dims)(dayofweeks_input)
    dayofweeks_reshape = Reshape((meta_embedding_dims,))(dayofweeks_embedding)

    minutes_input = Input(shape=(1,), name='minutes_input')
    minutes_embedding = Embedding(60, meta_embedding_dims)(minutes_input)
    minutes_reshape = Reshape((meta_embedding_dims,))(minutes_embedding)

    dayofyears_input = Input(shape=(1,), name='dayofyears_input')
    dayofyears_embedding = Embedding(366, meta_embedding_dims)(dayofyears_input)
    dayofyears_reshape = Reshape((meta_embedding_dims,))(dayofyears_embedding)
    
    merged = concatenate([titles_pooling, hours_reshape, dayofweeks_reshape, minutes_reshape, dayofyears_reshape])

    hidden_1 = Dense(256, activation='relu')(merged)
    hidden_1 = BatchNormalization()(hidden_1)

    main_output = Dense(1, activation='sigmoid', name='main_out')(hidden_1)
    
    model = Model(inputs=[titles_input,
                      hours_input,
                      dayofweeks_input,
                      minutes_input,
                      dayofyears_input], outputs=[main_output, aux_output])
    
#     model = Model(inputs=[titles_input], outputs=[aux_output])

    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model

In [18]:

model = build_model()

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
titles_input (InputLayer)       (None, 20)           0                                            
__________________________________________________________________________________________________
hours_input (InputLayer)        (None, 1)            0                                            
__________________________________________________________________________________________________
dayofweeks_input (InputLayer)   (None, 1)            0                                            
__________________________________________________________________________________________________
minutes_input (InputLayer)      (None, 1)            0                                            
__________________________________________________________________________________________________
dayofyears

In [13]:
seed(123)
split = 0.2

# returns randomized indices with no repeats
idx = sample(range(titles_tf.shape[0]), titles_tf.shape[0])

titles_tf = titles_tf[idx, :]
hours = hours[idx]
dayofweeks = dayofweeks[idx]
minutes = minutes[idx]
dayofyears_tf = dayofyears_tf[idx]
is_top_submission = is_top_submission[idx]

In [14]:
class MemoryCallback(Callback):
    def on_epoch_end(self, epoch, log={}):
        gc.collect()
        process = psutil.Process(os.getpid())
        print('\n memory: {} \n'.format(process.memory_info()))

In [15]:
model = Sequential()
model.add(Dense(1, input_shape=(maxlen,)))
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.


In [19]:
history = model.fit([titles_tf, hours, dayofweeks, minutes, dayofyears_tf], [is_top_submission, is_top_submission],
          batch_size=batch_size,
          epochs=epochs,
          validation_split=split,
          callbacks=[MemoryCallback()])
# history = model.fit([titles_tf], [is_top_submission],
#           batch_size=batch_size,
#           epochs=epochs,
#           validation_split=split,
#           callbacks=[MemoryCallback()])

Train on 12800 samples, validate on 3200 samples
Epoch 1/10
12736/12800 [============================>.] - ETA: 0s - loss: 8.6208 - main_out_loss: 0.7172 - aux_out_loss: 7.9036 - main_out_acc: 0.5158 - aux_out_acc: 0.5054
 memory: pmem(rss=824684544, vms=4748656640, shared=119640064, text=2322432, lib=0, data=1277263872, dirty=0) 

12800/12800 [==============================] - 9s 726us/sample - loss: 8.6258 - main_out_loss: 0.7170 - aux_out_loss: 7.9088 - main_out_acc: 0.5163 - aux_out_acc: 0.5051 - val_loss: 8.5882 - val_main_out_loss: 0.7001 - val_aux_out_loss: 7.8881 - val_main_out_acc: 0.4963 - val_aux_out_acc: 0.5066
Epoch 2/10
12672/12800 [============================>.] - ETA: 0s - loss: 8.5758 - main_out_loss: 0.6818 - aux_out_loss: 7.8940 - main_out_acc: 0.5607 - aux_out_acc: 0.5066
 memory: pmem(rss=825225216, vms=4748656640, shared=119640064, text=2322432, lib=0, data=1277304832, dirty=0) 

12800/12800 [==============================] - 7s 527us/sample - loss: 8.5870 - main